Buy and Hold Strategy

In [4]:
def buy_and_hold_strategy(prices, initial_cash=1_000_000):
    """
    Simple Buy & Hold backtest.

    Parameters
    ----------
    prices : array-like
        Time series of asset prices (list, Series, or numpy array).
    initial_cash : float, optional
        Starting capital. Default is 1,000,000.

    Returns
    -------
    results : dict
        {
            'portfolio_value': portfolio value over time,
            'returns': log returns of portfolio,
            'total_return': final return ratio
        }
    """
    S = np.array(prices)
    n = len(S)

    # Initial investment
    shares = initial_cash / S[0]
    portfolio_value = shares * S

    total_return = (S[-1] - S[0]) / S[0]
    print(f"Buy & Hold return: {total_return:.2%}")

    # Daily/periodic returns
    returns = np.insert(S[1:] / S[:-1], 0, 1)
    log_returns = np.log(returns)

    return {
        "portfolio_value": portfolio_value,
        "returns": log_returns,
        "total_return": total_return
    }


Mean Reverting Strategy

In [5]:
def mean_reversion_strategy(prices, initial_cash=1000000, time_window=10):
    """
    Simple mean reversion backtest.

    Parameters
    ----------
    prices : array-like
        Time series of asset prices (e.g., list or numpy array).
    initial_cash : float, optional
        Starting capital. Default is 1,000,000.
    time_window : int, optional
        Rolling window size for the moving average. Default is 10.

    Returns
    -------
    results : dict
        {
            'MRValue': portfolio value over time,
            'weights': position size in asset,
            'cash': remaining cash over time,
            'returns': log returns of portfolio,
            'total_return': final return ratio
        }
    """
    S = np.array(prices)
    n = len(S)

    cumsum = [0]
    ma = np.zeros(n)
    w = np.zeros(n)
    cash = np.zeros(n)

    cash[0] = initial_cash
    MRValue = [initial_cash, initial_cash]

    for i, x in enumerate(S[:-1], 0):
        cumsum.append(cumsum[i] + x)
        ma[i] = x
        if i >= time_window:
            moving_average = (cumsum[i] - cumsum[i - time_window]) / time_window
            ma[i] = moving_average

        # Decision logic
        if ma[i] == x:
            w[i + 1] = w[i]
            cash[i + 1] = cash[i]
        elif ma[i] > x:  # Buy signal
            w[i + 1] = cash[i] / x + w[i]
            cash[i + 1] = 0
        elif ma[i] < x:  # Sell signal
            cash[i + 1] = w[i] * x + cash[i]
            w[i + 1] = 0

        ma[i + 1] = S[-1]
        MRValue.append(cash[i + 1] + x * w[i + 1])

    mean_strategy = w * S + cash
    total_return = (mean_strategy[-1] - mean_strategy[0]) / mean_strategy[0]
    log_returns = [np.log(i / j) for i, j in zip(MRValue[1:], MRValue[:-1])]

    print(f"Mean reversion return: {total_return:.2%}")

    return {
        "MRValue": MRValue,
        "weights": w,
        "cash": cash,
        "returns": log_returns,
        "total_return": total_return
    }


Trend-following Strategy

In [6]:
def trend_following_strategy(prices, initial_cash=1_000_000, time_window=10):
    """
    Simple trend-following backtest.

    Parameters
    ----------
    prices : array-like
        Time series of asset prices (list, Series, or numpy array).
    initial_cash : float, optional
        Starting capital. Default is 1,000,000.
    time_window : int, optional
        Rolling window size for the moving average. Default is 10.

    Returns
    -------
    results : dict
        {
            'TFValue': portfolio value over time,
            'weights': position size in asset,
            'cash': remaining cash over time,
            'returns': log returns of portfolio,
            'total_return': final return ratio
        }
    """
    S = np.array(prices)
    n = len(S)

    cumsum = [0]
    ma = np.zeros(n)
    w = np.zeros(n)
    cash = np.zeros(n)

    cash[0] = initial_cash
    TFValue = [initial_cash, initial_cash]

    for i, x in enumerate(S[:-1], 0):
        cumsum.append(cumsum[i] + x)
        ma[i] = x
        if i >= time_window:
            moving_average = (cumsum[i] - cumsum[i - time_window]) / time_window
            ma[i] = moving_average

        # Trading logic
        if ma[i] == x:
            w[i + 1] = w[i]
            cash[i + 1] = cash[i]
        elif ma[i] < x:  # Buy signal (trend following)
            w[i + 1] = cash[i] / x + w[i]
            cash[i + 1] = 0
        elif ma[i] > x:  # Sell signal
            cash[i + 1] = w[i] * x + cash[i]
            w[i + 1] = 0

        ma[i + 1] = S[-1]
        TFValue.append(cash[i + 1] + x * w[i + 1])

    trend_strategy = w * S + cash
    total_return = (trend_strategy[-1] - trend_strategy[0]) / trend_strategy[0]
    log_returns = [np.log(i / j) for i, j in zip(TFValue[1:], TFValue[:-1])]

    print(f"Trend following strategy return: {total_return:.2%}")

    return {
        "TFValue": TFValue,
        "weights": w,
        "cash": cash,
        "returns": log_returns,
        "total_return": total_return
    }


Plotting the results

In [8]:
def plot_strategy_comparison(prices, buy_hold_result, trend_result, mean_result, title="Strategy Comparison"):
    """
    Plot portfolio values of Buy & Hold, Trend Following, and Mean Reversion strategies.

    Parameters
    ----------
    prices : array-like
        Original price series used for backtesting.
    buy_hold_result : dict
        Output of buy_and_hold_strategy().
    trend_result : dict
        Output of trend_following_strategy().
    mean_result : dict
        Output of mean_reversion_strategy().
    title : str, optional
        Plot title.
    """
    sns.set(context='paper', style='darkgrid', rc={'figure.facecolor': 'white'}, font_scale=1.5)
    
    T = np.arange(len(prices))
    seasize = (12.7, 8.27)
    fig, ax = plt.subplots(figsize=seasize)

    plt.title(title)
    plt.xlabel('Timestep')
    plt.ylabel('Wealth')

    # Plot each strategy’s portfolio value
    sns.lineplot(x=T, y=buy_hold_result['portfolio_value'], color='blue', label='Buy & Hold')
    sns.lineplot(x=T, y=trend_result['TFValue'], color='fuchsia', label='Trend Following')
    sns.lineplot(x=T, y=mean_result['MRValue'], color='green', label='Mean Reversion')

    plt.legend()
    plt.tight_layout()
    plt.show()
